<a href="https://colab.research.google.com/github/PERSONAL-PROJECTS-2000/SMART-STUDY-ASSISTANT/blob/main/SMART_STUDY_ASSISTANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-genai
%pip install streamlit
%pip install PyPDF2
%pip install python-pptx
%pip install python-docx
%pip install openpyxl
%pip install Pillow
%pip install pytesseract
%pip install pdf2image
!apt-get install -y tesseract-ocr poppler-utils > /dev/null 2>&1

In [ ]:
!mkdir -p stdast
!mkdir -p stdast/utils
!mkdir -p stdast/tabs

In [ ]:
with open('stdast/utils/APIs.py', 'w') as f1:
    f1.write('''
from google import genai
import streamlit as SL
class APIKeyManager:
    @staticmethod
    def get_current_key():
        if not SL.session_state.api_keys:
            return None
        return SL.session_state.api_keys[SL.session_state.current_key_index]
    @staticmethod
    def rotate_key():
        if len(SL.session_state.api_keys) > 1:
            SL.session_state.current_key_index = (SL.session_state.current_key_index + 1) % len(SL.session_state.api_keys)
            return True
        return False
def configure_gemini():
    key = APIKeyManager.get_current_key()
    if key:
        try:
            client = genai.Client(api_key=key)
            SL.session_state.client = client
            return client
        except Exception as e:
            return None
    return None
def call_gemini_with_retry(prompt, max_retries=None):
    if max_retries is None:
        max_retries = max(len(SL.session_state.api_keys), 1)
    for attempt in range(max_retries):
        try:
            client = configure_gemini()
            if not client:
                return "Please configure at least one API key in the sidebar."
            response = client.models.generate_content(
                model='gemini-2.0-flash-exp',
                contents=prompt
            )
            return response.text
        except Exception as e:
            error_str = str(e).lower()
            if "quota" in error_str or "limit" in error_str or "resource_exhausted" in error_str:
                if APIKeyManager.rotate_key():
                    continue
            if attempt == max_retries - 1:
                return f"Error: {str(e)}"
    return "All API keys exhausted. Please add more keys or try later."
''')

In [ ]:
with open('stdast/utils/DOCs.py', 'w') as f2:
    f2.write('''
import io
import os
from PIL import Image
from PyPDF2 import PdfReader
from pptx import Presentation
import docx
import openpyxl
import pytesseract
from pdf2image import convert_from_bytes
def extract_text_from_file(file):
    if file is None:
        return ""
    file_ext = os.path.splitext(file.name)[1].lower()
    try:
        if file_ext == '.txt':
            return file.read().decode('utf-8')
        elif file_ext == '.pdf':
            text = ""
            try:
                reader = PdfReader(io.BytesIO(file.read()))
                for page in reader.pages:
                    text += page.extract_text() + "\\n"
            except:
                pass
            if len(text.strip()) < 50:
                file.seek(0)
                images = convert_from_bytes(file.read())
                for img in images:
                    text += pytesseract.image_to_string(img) + "\\n"
            return text
        elif file_ext == '.docx' or file_ext == '.doc':
            doc = docx.Document(io.BytesIO(file.read()))
            return "\\n".join([para.text for para in doc.paragraphs])
        elif file_ext == '.pptx':
            prs = Presentation(io.BytesIO(file.read()))
            text = ""
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text += shape.text + "\\n"
            return text
        elif file_ext == '.xlsx':
            wb = openpyxl.load_workbook(io.BytesIO(file.read()))
            text = ""
            for sheet in wb.worksheets:
                for row in sheet.iter_rows(values_only=True):
                    text += " ".join([str(cell) for cell in row if cell]) + "\\n"
            return text
        elif file_ext in ['.jpg', '.jpeg', '.png']:
            img = Image.open(io.BytesIO(file.read()))
            return pytesseract.image_to_string(img)
    except Exception as e:
        return f"Error extracting text: {str(e)}"
    return ""
''')

In [ ]:
with open('stdast/utils/Theme.py', 'w') as f3:
    f3.write('''
import streamlit as SL
def apply_theme():
    if SL.session_state.theme == 'day':
        SL.markdown("""
        <style>
        .stApp {
            background-color: #f5f5f5;
            color: #000000;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: white;
            color: #000000;
        }
        </style>
        """, unsafe_allow_html=True)
    else:
        SL.markdown("""
        <style>
        .stApp {
            background-color: #1a1a1a;
            color: #e0e0e0;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: #2d2d2d;
            color: #e0e0e0;
        }
        </style>
        """, unsafe_allow_html=True)
''')

In [ ]:
with open('stdast/tabs/Chat.py', 'w') as f4:
    f4.write('''
import streamlit as SL
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
def render():
    SL.markdown("### Chat with AI")
    for message in SL.session_state.chat_history:
        with SL.chat_message(message["role"]):
            SL.write(message["content"])
    user_input = SL.chat_input("Ask me anything...")
    if user_input:
        SL.session_state.chat_history.append({"role": "user", "content": user_input})
        with SL.chat_message("user"):
            SL.write(user_input)
        with SL.chat_message("assistant"):
            response = call_gemini_with_retry(user_input)
            SL.write(response)
            SL.session_state.chat_history.append({"role": "assistant", "content": response})
''')

In [ ]:
with open('stdast/tabs/Summ.py', 'w') as f5:
    f5.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_summarize():
    SL.markdown("### Summarize Your Document")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sum_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sum_file")
    with col2:
        sum_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sum_topic")
    if SL.button("Summarize", type="primary", key="sum_btn"):
        if sum_file:
            with SL.spinner("Generating summary..."):
                text = extract_text_from_file(sum_file)
                prompt = f"Summarize the following document"
                if sum_topic:
                    prompt += f" focusing on the topic '{sum_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Summary", result, height=400, key="sum_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Exp.py', 'w') as f6:
    f6.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_explain():
    SL.markdown("### Explain Document Content")
    col1, col2 = SL.columns([2, 1])
    with col1:
        exp_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="exp_file")
    with col2:
        exp_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="exp_topic")
    SL.markdown("**Select Difficulty Level:**")
    col1, col2, col3 = SL.columns(3)
    levels = [("beginner", col1, "exp_beg"), ("intermediate", col2, "exp_int"), ("advanced", col3, "exp_adv")]
    for level, col, key in levels:
        with col:
            if SL.button(f"Explain for {level} level", type="primary", use_container_width=True, key=f"btn_{key}"):
                if exp_file:
                    with SL.spinner("Generating explanation..."):
                        text = extract_text_from_file(exp_file)
                        prompt = f"Explain the following document at a {level} level"
                        if exp_topic:
                            prompt += f" focusing on the topic '{exp_topic}'"
                        prompt += f":\\n\\n{text}"
                        result = call_gemini_with_retry(prompt)
                        SL.text_area("Explanation", result, height=400, key=key)
                else:
                    SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/KeyW.py', 'w') as f7:
    f7.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_keywords():
    SL.markdown("### Find and Explain Key Words")
    col1, col2 = SL.columns([2, 1])
    with col1:
        kw_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="kw_file")
    with col2:
        kw_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="kw_topic")
    if SL.button("Find and explain key words", type="primary", key="kw_btn"):
        if kw_file:
            with SL.spinner("Finding keywords..."):
                text = extract_text_from_file(kw_file)
                prompt = f"Identify all key terms/keywords from this document"
                if kw_topic:
                    prompt += f" related to '{kw_topic}'"
                prompt += f", and explain each one with examples:\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Keywords & Explanations", result, height=400, key="kw_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Sim.py', 'w') as f8:
    f8.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_similar():
    SL.markdown("### Find Similar Documents/Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sim_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sim_file")
    with col2:
        sim_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sim_topic")
    if SL.button("Find similar document/image", type="primary", key="sim_btn"):
        if sim_file:
            with SL.spinner("Searching for similar content..."):
                text = extract_text_from_file(sim_file)[:500]
                file_ext = os.path.splitext(sim_file.name)[1].lower()
                search_type = "similar image" if file_ext in ['.jpg', '.jpeg', '.png'] else "similar document or article"
                prompt = f"Search the web for a {search_type} containing information about: {text}"
                if sim_topic:
                    prompt += f" related to '{sim_topic}'"
                prompt += "\\n\\nProvide the link and a brief description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Similar Content", result, height=400, key="sim_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Links.py', 'w') as f9:
    f9.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_links():
    SL.markdown("### Find Related Web Resources")
    col1, col2 = SL.columns([2, 1])
    with col1:
        link_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="link_file")
    with col2:
        link_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="link_topic")
    if SL.button("Find links", type="primary", key="link_btn"):
        if link_file:
            with SL.spinner("Searching for web links..."):
                text = extract_text_from_file(link_file)[:300]
                prompt = f"Search the web and provide at least 5 links to documentation or information sources about: {text}"
                if link_topic:
                    prompt += f" focusing on '{link_topic}'"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Web Links", result, height=400, key="link_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/QA.py', 'w') as f10:
    f10.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_qa():
    SL.markdown("### Generate Answers to Questions")
    col1, col2 = SL.columns([2, 1])
    with col1:
        qa_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="qa_file")
    with col2:
        qa_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="qa_topic")
    if SL.button("Generate answers", type="primary", key="qa_btn"):
        if qa_file:
            with SL.spinner("Generating answers..."):
                text = extract_text_from_file(qa_file)
                prompt = f"Identify all questions in this document and provide detailed answers"
                if qa_topic:
                    prompt += f" in the context of '{qa_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Questions & Answers", result, height=400, key="qa_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Img.py', 'w') as f11:
    f11.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_image_search():
    SL.markdown("### Find Relevant Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        img_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="img_file")
    with col2:
        img_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="img_topic")
    if SL.button("Find image", type="primary", key="img_btn"):
        if img_file:
            with SL.spinner("Searching for images..."):
                text = extract_text_from_file(img_file)[:200]
                prompt = f"Search the web for an image that matches this description: {text}"
                if img_topic:
                    prompt += f" related to '{img_topic}'"
                prompt += "\\n\\nProvide the image link and description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Image Results", result, height=400, key="img_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/TimTab.py', 'w') as f12:
    f12.write('''
import streamlit as SL
import json
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
def render():
    SL.markdown("### Create Your Study Timetable")
    with SL.form("task_form"):
        col1, col2, col3, col4 = SL.columns(4)
        with col1:
            task_name = SL.text_input("Task Name", placeholder="e.g., Study Math")
        with col2:
            from_datetime = SL.text_input("From (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col3:
            to_datetime = SL.text_input("To (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col4:
            est_time = SL.text_input("Estimated Time (Optional)", placeholder="e.g., 2 hours")
        add_task = SL.form_submit_button("➕ Add Task")
        if add_task and task_name:
            SL.session_state.tasks.append({
                "name": task_name,
                "from": from_datetime,
                "to": to_datetime,
                "estimated": est_time
            })
            SL.success(f"Added task: {task_name}")
    if SL.session_state.tasks:
        SL.markdown("#### Current Tasks")
        for i, task in enumerate(SL.session_state.tasks):
            SL.write(f"{i+1}. **{task['name']}** | From: {task['from']} | To: {task['to']} | Est: {task['estimated']}")
        if SL.button("Clear All Tasks"):
            SL.session_state.tasks = []
            SL.rerun()
    total_time = SL.text_input("Estimated Total Time", placeholder="e.g., 8 hours")
    if SL.button("Generate Timetable", type="primary"):
        if SL.session_state.tasks:
            with SL.spinner("Generating timetable..."):
                task_str = json.dumps(SL.session_state.tasks, indent=2)
                prompt = f"""Create a detailed timetable based on these tasks:
{task_str}
Total estimated time available: {total_time}
Generate a realistic schedule in table format with task names and time allocations."""
                result = call_gemini_with_retry(prompt)
                SL.markdown("#### Generated Timetable")
                SL.text_area("Timetable", result, height=400)
        else:
            SL.warning("Please add at least one task first!")
''')

In [ ]:
with open('stdast/SmartStudyAssistant.py', 'w') as f:
    f.write('''
import streamlit as SL
import sys
sys.path.append('/content/stdast')
from utils.Theme import apply_theme
from tabs import Chat, Summ, Exp, KeyW, Sim, Links, QA, Img, TimTab
SL.set_page_config(
    page_title="AI Study Assistant",
    page_icon="🎓",
    layout="wide",
    initial_sidebar_state="expanded"
)
if 'api_keys' not in SL.session_state:
    SL.session_state.api_keys = []
if 'current_key_index' not in SL.session_state:
    SL.session_state.current_key_index = 0
if 'theme' not in SL.session_state:
    SL.session_state.theme = 'day'
if 'tasks' not in SL.session_state:
    SL.session_state.tasks = []
if 'chat_history' not in SL.session_state:
    SL.session_state.chat_history = []
if 'client' not in SL.session_state:
    SL.session_state.client = None
apply_theme()
with SL.sidebar:
    SL.markdown("### 🔑 API Configuration")
    api_key_inputs = []
    for i in range(10):
        key = SL.text_input(f"API Key {i+1}", type="password", key=f"api_key_{i}", placeholder="Enter Gemini API key")
        if key:
            api_key_inputs.append(key)
    if SL.button("Load API Keys"):
        SL.session_state.api_keys = [k.strip() for k in api_key_inputs if k.strip()]
        SL.session_state.current_key_index = 0
        SL.success(f"Loaded {len(SL.session_state.api_keys)} API key(s)")
    SL.markdown("---")
    SL.markdown("### 🎨 Theme")
    col1, col2 = SL.columns(2)
    with col1:
        if SL.button("☀️ Day"):
            SL.session_state.theme = 'day'
            SL.rerun()
    with col2:
        if SL.button("🌙 Night"):
            SL.session_state.theme = 'night'
            SL.rerun()
SL.title("🎓 AI-powered Smart Study Assistant")
tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9 = SL.tabs(["💬 Chat","📝 Summarize","📚 Explain","🔑 Key Words","🔍 Similar Content","🌐 Web Links","❓ Q&A","🖼️ Image Search","📅 Timetable"])
with tab1:
    Chat.render()
with tab2:
    Summ.render_summarize()
with tab3:
    Exp.render_explain()
with tab4:
    KeyW.render_keywords()
with tab5:
    Sim.render_similar()
with tab6:
    Links.render_links()
with tab7:
    QA.render_qa()
with tab8:
    Img.render_image_search()
with tab9:
    TimTab.render()
''')

In [ ]:
!touch stdast/__init__.py
!touch stdast/utils/__init__.py
!touch stdast/tabs/__init__.py

In [ ]:
!streamlit run stdast/SmartStudyAssistant.py & npx localtunnel --port 8501